This is the setup for the final project. The idea is to run this file once or as few times as possible. 
The only veriable you need to change are the breakpoint and chunksize, since the control how many rows the project will use.

In [1]:
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
import psycopg2
from pathlib import Path
from hf import replaceWithNull, standard_clean, hard_clean , time_clean, create_relationship, write_to_csv
import numpy as np
from cleantext import clean
from os.path import exists

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [2]:
startTid = time.time()

Data to Python to CSV.
In the codeblock below we are opening the large CSV file and running 100 round of 10.000 rows each for a total of 1mil rows. 
The result are then put into a pandas dataframe. The runtime is about 30min. 

In [3]:
file = Path("../Data/1mio-raw.csv")

# Code for reading the csv file and separating.
data = []
i = 0
breakpoint = 1000
chunksize = 1000


In [4]:
with open (file, 'r', encoding='utf-8') as reader: 
    reader = pd.read_csv(reader, chunksize=chunksize,quotechar='"', delimiter=',')
    for chunk in reader:
        replaceWithNull(chunk)
        for column in chunk.columns:
            chunk[column] = chunk[column].astype(str)
            chunk[column] = chunk[column].str.replace("\n", "")
            chunk[column] = chunk[column].replace("nan", "null")
        data.append(chunk)
        i += 1
        if i >= breakpoint:
            break           
reader.close()
data = pd.concat(data)

# Converting to a pandas dataframe. 
df = pd.DataFrame(data)   
df.columns = ['None', 'Id', 'Domain', 'Type', 'Url', 'Content', 
                                   'Scraped_at', 'Inserted_at', 'Updated_at', 'Title', 
                                   'Authors', 'Keywords', 'Meta_keywords', 
                                   'Meta_description', 'Tags', 'Summary', 'Source']

Setup of SQL 

This is our two veriable where we store our SQL commands. First of we drop all of the  tables and we then create them. We used normalization and wrote our database in BCNF to avoid redundancy and exponentially large tables. We left a lot of functional dependencys in the articles table but chose not to divide it into smaller tables because we prioritised simplification of queries 


Belows we are setting up a connection to our local SQL server and first droping the tables (if any) and then creating the tables. 

In [5]:
#Droper Tables hvis de eksitere 
drop_tables = (""" DROP TABLE IF EXISTS article, url, author, keyword, meta_keyword, tag, article_tag, article_author, article_keyword, article_meta_keyword, Wiki""") 

#Samling af instruktioner til at lave flere tables
create_fake_news_corpus_tables =  ("""
    CREATE TABLE IF NOT EXISTS article(
    id VARCHAR(256) PRIMARY KEY,
    type VARCHAR(50),
    content VARCHAR,  
    scraped_at DATE,
    inserted_at DATE,
    updated_at DATE,
    title VARCHAR(250),
    meta_description VARCHAR,
    summary VARCHAR(100),
    source VARCHAR(50)
    );

    CREATE TABLE IF NOT EXISTS url(
    id VARCHAR(50),
    article_id VARCHAR(50),
    sld VARCHAR(50),
    tld VARCHAR(50),
    slug VARCHAR,
    ssl VARCHAR(3),
    PRIMARY KEY(id, article_id),
    FOREIGN KEY (article_id) REFERENCES article(id)
    );

    CREATE TABLE IF NOT EXISTS author(
    id VARCHAR(50) PRIMARY KEY, 
    name VARCHAR
    );
    
    
    CREATE TABLE IF NOT EXISTS article_author(
    article_id VARCHAR(50),
    author_id VARCHAR(50),
    PRIMARY KEY (article_id, author_id),
    FOREIGN KEY (article_id) REFERENCES article(id),
    FOREIGN KEY (author_id) REFERENCES author(id)
    );
    
    CREATE TABLE IF NOT EXISTS keyword(
    id VARCHAR(50) PRIMARY KEY, 
    label VARCHAR(50)
    );

       
    CREATE TABLE IF NOT EXISTS article_keyword(
    article_id VARCHAR(50),
    keyword_id VARCHAR(50),
    PRIMARY KEY (article_id, keyword_id),
    FOREIGN KEY (article_id) REFERENCES article(id),
    FOREIGN KEY (keyword_id) REFERENCES keyword(id)
    );
    
    
    CREATE TABLE IF NOT EXISTS meta_keyword(
    id VARCHAR(50) PRIMARY KEY, 
    label VARCHAR
    );

    CREATE TABLE IF NOT EXISTS article_meta_keyword( 
    article_id VARCHAR(50),
    meta_keyword_id VARCHAR(50),
    PRIMARY KEY (article_id, meta_keyword_id),
    FOREIGN KEY (article_id) REFERENCES article(id),
    FOREIGN KEY (meta_keyword_id) REFERENCES meta_keyword(id)
    );
    
    
    CREATE TABLE IF NOT EXISTS tag(
    id VARCHAR(50) PRIMARY KEY, 
    label VARCHAR
    );
    
    
    CREATE TABLE IF NOT EXISTS article_tag( 
    article_id VARCHAR(50),
    tag_id VARCHAR(50),
    PRIMARY KEY (article_id, tag_id),
    FOREIGN KEY (article_id) REFERENCES article(id),
    FOREIGN KEY (tag_id) REFERENCES tag(id)
    );
    """
)

alter_tables = ("""
    ALTER TABLE article
    ALTER COLUMN type
    TYPE VARCHAR(50)
    USING
        CASE type
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE type::VARCHAR(50)
        END;

    ALTER TABLE article
    ALTER COLUMN summary
    TYPE VARCHAR(100)
    USING
        CASE summary
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE summary::VARCHAR(100)
        END;

    ALTER TABLE article
    ALTER COLUMN source
    TYPE VARCHAR(50)
    USING
        CASE source
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE source::VARCHAR(50)
        END;

    ALTER TABLE author
    ALTER COLUMN name
    TYPE VARCHAR
    USING
        CASE name
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE name::VARCHAR
        END;

    ALTER TABLE keyword
    ALTER COLUMN label
    TYPE VARCHAR
    USING
        CASE label
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE label::VARCHAR
        END;

    ALTER TABLE meta_keyword
    ALTER COLUMN label
    TYPE VARCHAR
    USING
        CASE label
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE label::VARCHAR
        END;

    ALTER TABLE tag
    ALTER COLUMN label
    TYPE VARCHAR
    USING
        CASE label
            WHEN 'null' THEN NULL
            WHEN '' THEN NULL
            ELSE label::VARCHAR
        END;
""")

Cleaning the data

In [6]:
# Cleaning data
df["Authors"] = df["Authors"].str.replace("America, There", "America There")

df["Title"] = standard_clean(df["Title"])
df["Meta_description"] = standard_clean(df["Meta_description"])
df["Summary"] = standard_clean(df["Summary"])
df["Source"] = standard_clean(df["Source"])

df["Scraped_at"] = time_clean(df["Scraped_at"])
df["Inserted_at"] = time_clean(df["Inserted_at"])
df["Updated_at"] = time_clean(df["Updated_at"])


df["Content"] = df["Content"].astype(str)
df["Content"] = hard_clean(df["Content"])

Creating all dataframes

In [7]:
# Drop duplicates and invalid ID's
df.drop_duplicates(keep='first', inplace=True)
df = df[df["Id"].str.isnumeric()]

# Create relationships for all entities having a direct relationship e.g. article to any other entity
df_tags, df_article_tags= create_relationship(df, 'Tags', "tag_id", "label")
df_keywords, df_article_keywords = create_relationship(df, 'Keywords', "keyword_id", "label")
df_authors, df_article_authors = create_relationship(df, 'Authors', "author_id", "name")
df_meta_keywords, df_article_meta_keywords = create_relationship(df, 'Meta_keywords', "meta_keyword_id", "label")

# Creating the url dataframe
df_url = df[["Id", "Url", "Domain"]]
df_url["Url"] = df_url["Url"].str.replace("www.", "")
df_url["Domain"] = df_url["Domain"].str.replace("www.", "")
df_url["Protocol"] = df_url["Url"].str.split("://", 1, expand=True)[0]
df_url[["sld", "tld"]] = df_url["Domain"].str.split(".", 1, expand=True)
df_url["slug"] = df_url["Url"].str.split("/", 3, expand=True)[3]
df_url = df_url.assign(url_id=(df_url["Url"]).astype('category').cat.codes)
df_url.rename(columns={"Id": "article_id", "url_id": "id"}, inplace=True)
df_url.drop(columns="Domain", axis=1)
df_url = df_url.assign(ssl=(df_url["Protocol"]).astype('category').cat.codes)
df_url["ssl"] = df_url["ssl"].astype(str)
df_url["ssl"] = df_url["ssl"].str.replace("0", "No")
df_url["ssl"] = df_url["ssl"].str.replace("1", "Yes")
df_url = df_url[["id", "article_id", "sld", "tld", "slug", "ssl"]]
df_url["slug"] = standard_clean(df_url["slug"])
df_url["slug"] = hard_clean(df_url["slug"])


df_authors["name"] = df_authors["name"].replace(np.nan, "")
df_authors["name"] = df_authors["name"].str.replace("nan", "")
df_keywords["label"] = df_keywords["label"].replace(np.nan, "")
df_keywords["label"] = df_keywords["label"].str.replace("nan", "")
df_meta_keywords["label"] = df_meta_keywords["label"].replace(np.nan, "")
df_meta_keywords["label"] = df_meta_keywords["label"].str.replace("nan", "")
df_tags["label"] = df_tags["label"].replace(np.nan, "")
df_tags["label"] = df_tags["label"].str.replace("nan", "")

# Creating the article dataframe
df_articles = df[["Id", "Type", "Content", "Scraped_at", "Inserted_at", "Updated_at", "Title", "Meta_description", "Summary", "Source"]]

Segmenting the data and writing it into several CSV files. The run time here is more or less equally to the reading of the data.

In [8]:
# Writing entities to csv
write_to_csv(df_articles, "article")
write_to_csv(df_url, "url")
write_to_csv(df_authors, "author")
write_to_csv(df_keywords, "keyword")
write_to_csv(df_meta_keywords, "meta_keyword")
write_to_csv(df_tags, "tag")

# Writing relationships to csv
write_to_csv(df_article_authors, "article_author")
write_to_csv(df_article_keywords, "article_keyword")
write_to_csv(df_article_meta_keywords, "article_meta_keyword")
write_to_csv(df_article_tags, "article_tag")

In this block of code we are copying the data from the CSV files into the SQL database

In [9]:
# Connect to an existing database
conn = None
try:
    conn = psycopg2.connect(#database="public",
                        port = "5432",
                        host="localhost", 
                        user="postgres", 
                        password="1234")
except psycopg2.DatabaseError as e: 
        print (f'Error {e}')
        sys.exit(1)

# Open a cursor to perform database operations
cur = conn.cursor()
cur.execute(drop_tables)
cur.execute(create_fake_news_corpus_tables)

# Execute a command: this creates a new table

with open('article.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "article", columns=('id','type','content','scraped_at','inserted_at','updated_at','title','meta_description','summary', 'source'), sep=",")

with open('url.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "url", columns=('id', "article_id", "sld", "tld", "slug", "ssl"), sep=",")

with open('author.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "author", columns=('id', "name"), sep=",")

with open('keyword.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "keyword", columns=('id', "label"), sep=",")

with open('meta_keyword.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "meta_keyword", columns=('id', "label"), sep=",")

with open('tag.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "tag", columns=('id', "label"), sep=",")

with open('article_author.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "article_author", columns=('article_id', "author_id"), sep=",")

with open('article_keyword.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "article_keyword", columns=('article_id', "keyword_id"), sep=",")

with open('article_meta_keyword.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "article_meta_keyword", columns=('article_id', "meta_keyword_id"), sep=",")

with open('article_tag.csv', "r", encoding ='utf-8') as f:
        next(f)
        cur.copy_from(f, "article_tag", columns=('article_id', "tag_id"), sep=",")

#with open('wiki.csv', "r", encoding ='utf-8') as f:
#        next(f)
#        cur.copy_from(f, "wiki", columns=('title', 'date', 'content'), sep=",")

cur.execute(alter_tables)

# Proff that the database has the required number of rows
cur.execute("""SELECT COUNT(*) FROM article""")
f = cur.fetchall()
print ("Number of rows in the database: " , f)
# Make the changes to the database persistent
conn.commit()

# Close communication with the database
cur.close()
conn.close()

Number of rows in the database:  [(998937,)]


In this block of code we are copying the data from the CSV files into the SQL database

In [10]:
slutTid = time.time()
kage2 = slutTid-startTid
print(kage2)
print("Tid i min:" ,kage2/60)

1272.5962235927582
Tid i min: 21.2099370598793
